In [71]:
# Import the must library
import tweepy 
import csv
import pandas as pd
from textblob import TextBlob
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
#input the auth key to connect tweepy api, print result if connected sucessfully, 




CONSUMER_KEY = 'hy2RNcQ3AkHlOYwn3jR0mIn21'
CONSUMER_SECRET = 'TPr76aYBC0fkbLs5AVDwoUPWf8o0afm5UzimLEogHHqXUaZCmV'
OAUTH_TOKEN = '1095973961475731457-brMqEws4krkDvl5weHlfllqHkREjPl'
OAUTH_TOKEN_SECRET = 'vDIcQkJsTohzfSmRDqj38NDYvlSTC1geML95vb4sfn43k'
try:
    auth = tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
    auth.set_access_token(OAUTH_TOKEN,OAUTH_TOKEN_SECRET)
    api = tweepy.API(auth,wait_on_rate_limit=True)
    print("connected ",api)
except:
    print("Error: Authentication Failed") 
    
def tweets_crawling(hashtag,tweet_mode,lang,result_type,pagecount,items):
    df = pd.DataFrame()
    msgs = []
    msg =[]
    for tweet in tweepy.Cursor(api.search,q=hashtag,tweet_mode=tweet_mode,lang=lang,result_type=result_type,count=pagecount).items(items):
        if 'retweeted_status' in dir(tweet):
            text=tweet.retweeted_status.full_text
        else:
            text=tweet.full_text
        msg=[tweet.user.id,tweet.user.name,tweet.user.location,tweet.user.friends_count,tweet.created_at,tweet.retweet_count,tweet.favorite_count,text]
        msgs.append(msg)
    df=pd.DataFrame(msgs)
    df.columns=['user_id','user_name','user_location','user_friends_count','created_time','retweet_count','favourite_count','original_text']
    return df

def extract_hashtag(text):
    hashtag=[]
    for i in text:
        a=re.findall(r"#(\w+)", i)
        hashtag.append([i.capitalize() for i in a])
    return hashtag

def data_clean(text):
    p=re.compile(r'[-,$()#+&*]')
    urlclean=re.compile(r'https://[a-zA-Z0-9.?/&=:]*',re.S)
    tagclean=re.compile(r'#[a-zA-Z0-9.?/&=:_]*',re.S)
    userclean=re.compile(r'@[a-zA-Z0-9.?/&=:_]*',re.S)
    cleaned_text=[]

    for i in text:
        nourl=re.sub(urlclean,"",i)
        nouser=re.sub(userclean,"",nourl)
        notag=re.sub(tagclean,"",nouser)
        clean =notag.replace('\n',"")
        cleaned_text.append(clean)
    return cleaned_text
    
    

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...


connected  <tweepy.api.API object at 0x00000233C4F42E88>


In [83]:

df = tweets_crawling(hashtag='#vegan',tweet_mode='extended',lang='en',result_type='mixed',pagecount=100,items=100)
df.to_csv('vegancrawl.csv',header=True,index=True,encoding='utf_8_sig')
df['hashtag'] = extract_hashtag(df['original_text'])
df['cleaned_text'] = data_clean(df['original_text'])
 
    


In [78]:
df=pd.read_csv('vegancrawl.csv',index_col=None)
df['hashtag'] = extract_hashtag(df['original_text'])
df['cleaned_text'] = data_clean(df['original_text'])
df.head(5)

,Unnamed: 0,user_id,user_name,user_location,user_friends_count,created_time,retweet_count,favourite_count,original_text,hashtag,cleaned_text
0,0,56814425,espialtoday,"Houston, Texas",2314,2020-01-17 17:16:27,1466,0,"The feathers needed for bedding, jackets etc. ...","[Govegan, Vegan, Animalrights]","The feathers needed for bedding, jackets etc. ..."
1,1,599677362,Ruby #EndBSL,Deepest Dorset,2517,2020-01-17 17:16:27,49,0,Are you one less person harming animals?⁣⁠\n⁣⁠...,[Vegan],Are you one less person harming animals?⁣⁠⁣⁠Re...
2,2,1013711024,Luci 😈,UK,4965,2020-01-17 17:16:24,151,0,🌟Follow + RT🌟To #win INTRODUCING our 100% NATU...,"[Win, Crueltyfree, Vegan, Paraffinfree, Amazon...",🌟Follow + RT🌟To INTRODUCING our 100% NATURAL ...
3,3,939963727813775361,Mellamigo,Germany,51,2020-01-17 17:16:21,108,0,How adorable! See this lucky rescued pig 🐷 enj...,"[Vegan, Fridaythoughts, Fridayfeeling]",How adorable! See this lucky rescued pig 🐷 enj...
4,4,2285825144,Mummy Hasson,United Kingdom,3793,2020-01-17 17:16:04,151,0,🌟Follow + RT🌟To #win INTRODUCING our 100% NATU...,"[Win, Crueltyfree, Vegan, Paraffinfree, Amazon...",🌟Follow + RT🌟To INTRODUCING our 100% NATURAL ...
...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,2256664812,MRX,New Jersey,212,2020-01-15 00:33:20,1,0,Awesome DYI Birthday Cupcakes for @harpuranne ...,"[Glutenfree, Vegan, Yummmy]",Awesome DYI Birthday Cupcakes for from Pleas...
19996,19996,819697568087044096,USBlog Retweets,United States,1747,2020-01-15 00:33:15,4,0,"Classic hummus is simple and easy to make, but...","[Vegan, Veganrecipe, Worldcuisine, Blogger, Th...","Classic hummus is simple and easy to make, but..."
19997,19997,225278849,Life’s too Short to Not,"Queensland, Australia",2339,2020-01-15 00:33:14,681,0,😭 HOW MUCH ANIMAL SUFFERING IS NECESSARY BEFOR...,"[Climatecrisis, Veganuary, Veganuary2020, Vega...",😭 HOW MUCH ANIMAL SUFFERING IS NECESSARY BEFOR...
19998,19998,2256664812,MRX,New Jersey,212,2020-01-15 00:32:59,1,0,"My happy place, all you can eat sushi. 🙌 #vega...","[Vegan, Plantbased]","My happy place, all you can eat sushi. 🙌"


In [80]:
textblob_SA=[]
textblob_poly=[]
for i in df["cleaned_text"]:
    analsysis=TextBlob(i)
    textblob_SA.append(analsysis.sentiment.polarity)
    if analsysis.sentiment.polarity>=0.1:
        textblob_poly.append("Positive")
    elif analsysis.sentiment.polarity<=-0.1:
        textblob_poly.append("Negative")
    else:
        textblob_poly.append("Neutral")
df["textblob_SA"]=textblob_SA
df["textblob_poly"]=textblob_poly
df.head(5)

,Unnamed: 0,user_id,user_name,user_location,user_friends_count,created_time,retweet_count,favourite_count,original_text,hashtag,cleaned_text,textblob_SA,textblob_poly
0,0,56814425,espialtoday,"Houston, Texas",2314,2020-01-17 17:16:27,1466,0,"The feathers needed for bedding, jackets etc. ...","[Govegan, Vegan, Animalrights]","The feathers needed for bedding, jackets etc. ...",0.117188,Positive
1,1,599677362,Ruby #EndBSL,Deepest Dorset,2517,2020-01-17 17:16:27,49,0,Are you one less person harming animals?⁣⁠\n⁣⁠...,[Vegan],Are you one less person harming animals?⁣⁠⁣⁠Re...,-0.208333,Negative
2,2,1013711024,Luci 😈,UK,4965,2020-01-17 17:16:24,151,0,🌟Follow + RT🌟To #win INTRODUCING our 100% NATU...,"[Win, Crueltyfree, Vegan, Paraffinfree, Amazon...",🌟Follow + RT🌟To INTRODUCING our 100% NATURAL ...,0.123864,Positive
3,3,939963727813775361,Mellamigo,Germany,51,2020-01-17 17:16:21,108,0,How adorable! See this lucky rescued pig 🐷 enj...,"[Vegan, Fridaythoughts, Fridayfeeling]",How adorable! See this lucky rescued pig 🐷 enj...,0.409722,Positive
4,4,2285825144,Mummy Hasson,United Kingdom,3793,2020-01-17 17:16:04,151,0,🌟Follow + RT🌟To #win INTRODUCING our 100% NATU...,"[Win, Crueltyfree, Vegan, Paraffinfree, Amazon...",🌟Follow + RT🌟To INTRODUCING our 100% NATURAL ...,0.123864,Positive


In [82]:
nltk_vsa=[]
nltk_poly=[]
sid=SentimentIntensityAnalyzer()
for i in df["cleaned_text"]:
    nltk_vsa.append(sid.polarity_scores(i)['compound'])
    if sid.polarity_scores(i)['compound']>=0.1:
        nltk_poly.append("Positive")
    elif sid.polarity_scores(i)['compound']<=-0.1:
        nltk_poly.append("Negative")
    else:
        nltk_poly.append("Neutral")
        
df["nltk_vsa"]=nltk_vsa
df["nltk_poly"]=nltk_poly
df.head(5)

,Unnamed: 0,user_id,user_name,user_location,user_friends_count,created_time,retweet_count,favourite_count,original_text,hashtag,cleaned_text,textblob_SA,textblob_poly,nltk_vsa,nltk_poly
0,0,56814425,espialtoday,"Houston, Texas",2314,2020-01-17 17:16:27,1466,0,"The feathers needed for bedding, jackets etc. ...","[Govegan, Vegan, Animalrights]","The feathers needed for bedding, jackets etc. ...",0.117188,Positive,0.2579,Positive
1,1,599677362,Ruby #EndBSL,Deepest Dorset,2517,2020-01-17 17:16:27,49,0,Are you one less person harming animals?⁣⁠\n⁣⁠...,[Vegan],Are you one less person harming animals?⁣⁠⁣⁠Re...,-0.208333,Negative,-0.5594,Negative
2,2,1013711024,Luci 😈,UK,4965,2020-01-17 17:16:24,151,0,🌟Follow + RT🌟To #win INTRODUCING our 100% NATU...,"[Win, Crueltyfree, Vegan, Paraffinfree, Amazon...",🌟Follow + RT🌟To INTRODUCING our 100% NATURAL ...,0.123864,Positive,0.9066,Positive
3,3,939963727813775361,Mellamigo,Germany,51,2020-01-17 17:16:21,108,0,How adorable! See this lucky rescued pig 🐷 enj...,"[Vegan, Fridaythoughts, Fridayfeeling]",How adorable! See this lucky rescued pig 🐷 enj...,0.409722,Positive,0.9870,Positive
4,4,2285825144,Mummy Hasson,United Kingdom,3793,2020-01-17 17:16:04,151,0,🌟Follow + RT🌟To #win INTRODUCING our 100% NATU...,"[Win, Crueltyfree, Vegan, Paraffinfree, Amazon...",🌟Follow + RT🌟To INTRODUCING our 100% NATURAL ...,0.123864,Positive,0.9066,Positive
